
<a href="https://blog.cloudcommander.net" target="_parent"><img src="https://raw.githubusercontent.com/cloud-commander/hexoblog/master/cloud.png" alt="Visit my Blog">
</a>
<br> 
# <span style="font-family:Didot; font-size:3em;"> Cloud Commander </span>


<a href="https://colab.research.google.com/github/cloud-commander/face-mask-detection/blob/master/1_Prepare_Data_Annotate_Images_Part1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"></a>
&nbsp;&nbsp;&nbsp;&nbsp;
<a href="https://github.com/cloud-commander/face-mask-detection/blob/master/1_Prepare_Data_Annotate_Images_Part1.ipynb" target="_parent"><img src="https://img.shields.io/static/v1?logo=GitHub&label=&color=333333&style=flat&message=View%20on%20GitHub" alt="View in GitHub"></a>



## Automatic Face Image Annotation  ##

We start off the data preparation phase by pre-processing images in our dataset for the unmasked category.

We have a selection of facial images and we need to draw bounding boxes around them. We could do that manually using a tool such as [labelImg](https://github.com/tzutalin/labelImg) however that would be slow, tedious and unnecessary for our purposes.

Instead we will use a facial detection algorithm (Haar cascade) to automatically detect faces and generate an accompanying XML file for each image with the coordinates of the face. Now this method is far from foolproof as it only works properly with full frontal images of faces but its a good starting point.



### Import required libraries

In [0]:
#!pip install lxml
#!pip install opencv-python
from lxml import etree as ET

import os
import cv2
import numpy as np
import glob
from pathlib import Path
import shutil

from PIL import Image, ImageFile

### Prepare dataset ###

#### Connect to Google Drive

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


#### Copy dataset from Google Drive and extract


In [0]:
!cp /content/gdrive/My\ Drive/Datasets/unmasked_photos.tar.gz /content/

In [0]:
!mkdir /content/dataset/ && tar -C /content/dataset/ -xvf unmasked_photos.tar.gz

unmasked_photos/
unmasked_photos/QDNP2YSTZQ.jpg
unmasked_photos/DNTI3XNN1M.jpg
unmasked_photos/B9T63OSE3L.jpg
unmasked_photos/TMLUG56C5G.jpg
unmasked_photos/I2MGQU4YQG.jpg
unmasked_photos/YQBO424QKU.jpg
unmasked_photos/US1K4RZH7S.jpg
unmasked_photos/3BWYH1F0QL.jpg
unmasked_photos/6C8ANK9IA1.jpg
unmasked_photos/PQ8B522YVY.jpg
unmasked_photos/RU9DJMORBI.jpg
unmasked_photos/5TVFA7WXE6.jpg
unmasked_photos/W1HHYF6D92.jpg
unmasked_photos/O1QLRME2TO.jpg
unmasked_photos/MY5B7UD8BR.jpg
unmasked_photos/H4RHQ1KGBP.jpg
unmasked_photos/NL0E80Q1MD.jpg
unmasked_photos/202K2MKRWE.jpg
unmasked_photos/TINHNEVOL1.jpg
unmasked_photos/MLCG6N8CXX.jpg
unmasked_photos/9E5FII1U9P.jpg
unmasked_photos/PSI1IB1QR9.jpg
unmasked_photos/I7EAH576PP.jpg
unmasked_photos/JR02CQDGUE.jpg
unmasked_photos/QLPQP937TK.jpg
unmasked_photos/F394Q3955Y.jpg
unmasked_photos/BOP21MV1KF.jpg
unmasked_photos/YK1MLBA8T2.jpg
unmasked_photos/GV10QOE4IN.jpg
unmasked_photos/LS31D1IGD3.jpg
unmasked_photos/PL40A1KEVB.jpg
unmasked_photos/PGNNOR

In [0]:
!ls

content  gdrive  LICENSE.txt  README.txt  sample_data  unmasked_photos.tar.gz


#### Set image directory ####

In [0]:
dataset_dir = "dataset/unmasked_photos"

#### Download required dependency 

In [0]:
!wget https://raw.githubusercontent.com/cloud-commander/face-mask-detection/master/utils/lbpcascade_frontalface_improved.xml

--2020-06-04 11:24:31--  https://raw.githubusercontent.com/cloud-commander/face-mask-detection/master/utils/lbpcascade_frontalface_improved.xml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 54039 (53K) [text/plain]
Saving to: ‘lbpcascade_frontalface_improved.xml’

lbpcascade_frontalf 100%[===================>]  52.77K  --.-KB/s    in 0.03s   

2020-06-04 11:24:32 (2.02 MB/s) - ‘lbpcascade_frontalface_improved.xml’ saved [54039/54039]



In [0]:
path = os.getcwd()
filename = path + "/" + dataset_dir
face_cascade = cv2.CascadeClassifier('lbpcascade_frontalface_improved.xml')
dataset_labels_folder = dataset_dir + "-labels"
dataset_labels_path = os.getcwd()+ "/"+ dataset_labels_folder 
print(dataset_labels_path)
os.mkdir(dataset_labels_folder)


/content/dataset/unmasked_photos-labels


In [0]:
for subdir, dirs, files in os.walk(filename):
    for file in files:

        
        img_path=os.path.join(subdir, file)
        img_name=os.path.basename(img_path)

        if img_path.lower().endswith(('.png', '.jpg', '.jpeg')):

            img = cv2.imread(img_path)
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            h,w,bpp = np.shape(img)
            imgp = Image.open(img_path)
#            b, landmarks = detect_faces(imgp)
            faces = face_cascade.detectMultiScale(gray, 1.3, 5)


#            g=show_bboxes(imgp, b, landmarks)
            #print(len(faces))
            if len(faces) == 1 :
                try :

                    for bounding_boxes in faces:
                        face = img[int(bounding_boxes[1]):int(bounding_boxes[3]),
                        int(bounding_boxes[0]):int(bounding_boxes[2])]

                        
                        subdir_path, subdir_name = os.path.split(subdir)

                        root = ET.Element("annotation",verified="yes")
                        ET.SubElement(root, "folder").text=filename

                        ET.SubElement(root, "filename").text = img_name
                        ET.SubElement(root, "path").text = img_path

                        source=ET.SubElement(root, "source")
                        ET.SubElement(source, "database").text = "unknown"

                        size=ET.SubElement(root, "size")
                        ET.SubElement(size, "width").text = str(w)
                        ET.SubElement(size, "height").text = str(h)
                        ET.SubElement(size, "depth").text = str(bpp)

                        ET.SubElement(root, "segmented").text = "0"

                        obj=ET.SubElement(root, "object")
                        ET.SubElement(obj, "name").text = subdir_name
                        ET.SubElement(obj, "pose").text = "Unspecified"
                        ET.SubElement(obj, "truncated").text = "0"
                        ET.SubElement(obj, "difficult").text = "0"

                        box=ET.SubElement(obj, "bndbox")
                        ET.SubElement(box, "xmin").text = str(int(bounding_boxes[0]))
                        ET.SubElement(box, "ymin").text = str(int(bounding_boxes[1]))
                        ET.SubElement(box, "xmax").text = str(int(bounding_boxes[2])+int(bounding_boxes[1]))
                        ET.SubElement(box, "ymax").text = str(int(bounding_boxes[3]))

                        tree = ET.ElementTree(root)
                        tree.write(os.path.join(dataset_labels_path, os.path.splitext(img_name)[0] + '.xml'))
                        #tree.write(os.path.join(filename +"-labels", os.path.splitext(img_name)[0] + '.xml'))
                        
                except RuntimeError :
                    with open("delete.txt", "a") as myfile:
                        myfile.write(img_path+"\n")
                	#print(img_path)

                else :
                    with open("delete.txt", "a") as myfile:
                            myfile.write(img_path+"\n")
                    #print('no face')

### Save Dataset

In [0]:
source1 = filename + "/"
source2 = dataset_labels_path + "/"

destination = os.getcwd() + "/training_demo"+ "/images"
if not os.path.exists(destination):
    os.makedirs(destination)


def move_files(source,destination):
    files = os.listdir(source)
    for f in files:
        shutil.move(source+f, destination)


move_files(source1,destination)
move_files(source2,destination)

# deletes source directories, use as needed
os.rmdir(source2)
os.rmdir(source1)

### Compress Dataset (Optional) ###

Follow this step if you wish to archive your dataset for future use

In [33]:
!tar -zcvf unmasked-dataset.tar.gz /content/training_demo

tar: Removing leading `/' from member names
/content/training_demo/
/content/training_demo/images/
/content/training_demo/images/VZGLQ059C1.jpg
/content/training_demo/images/9V1Y7JYQNF.xml
/content/training_demo/images/40RBMPI04G.jpg
/content/training_demo/images/K9PFT0A194.xml
/content/training_demo/images/WAKUO7V02E.xml
/content/training_demo/images/KA4FMGBO8H.xml
/content/training_demo/images/99XOH68P4O.jpg
/content/training_demo/images/9VFXN07J13.xml
/content/training_demo/images/GMD7MLC7N3.jpg
/content/training_demo/images/JPDKU084ZK.jpg
/content/training_demo/images/J9KV0D4ZLL.xml
/content/training_demo/images/11OTO6P279.jpg
/content/training_demo/images/IQ6FEAIL9Y.xml
/content/training_demo/images/QPHCREQRTG.jpg
/content/training_demo/images/F0ZN6JPDPY.jpg
/content/training_demo/images/J7R2EXRJ3E.jpg
/content/training_demo/images/X1F14CS70T.xml
/content/training_demo/images/DR66FB32BA.jpg
/content/training_demo/images/57BKPJZMG0.xml
/content/training_demo/images/01P734HDO0.jpg
/

In [0]:
!cp /content/unmasked-dataset.tar.gz /content/gdrive/My\ Drive/Datasets/